In [122]:
import findspark as fds
fds.init()

from pyspark.sql import SparkSession

# Starts new spark session
spark = SparkSession.builder.appName("DS_Final_Project").getOrCreate()

In [123]:
# Loads the data csv
hotels = spark.read.csv('Hotels_data_Changed.csv',inferSchema=True,header=True)

hotels.show()

+---+-----------+-------------------+-------------------+----+--------------+--------------+-------------+---------------+--------------------+-----------+--------------------+---------+------------+------------------+
|_c0|Snapshot ID|      Snapshot Date|       Checkin Date|Days|Original Price|Discount Price|Discount Code|Available Rooms|          Hotel Name|Hotel Stars|             DayDiff|  WeekDay|DiscountDiff|      DiscountPerc|
+---+-----------+-------------------+-------------------+----+--------------+--------------+-------------+---------------+--------------------+-----------+--------------------+---------+------------+------------------+
|  0|          1|2015-07-17 00:00:00|2015-08-12 00:00:00|   5|          1178|          1040|            1|              6|Best Western Plus...|          3|26 days 00:00:00....|Wednesday|         138|11.714770797962649|
|  1|          1|2015-07-17 00:00:00|2015-08-19 00:00:00|   5|          1113|           982|            1|              8|Be

In [124]:
import pyspark.sql.functions as f
from pyspark.sql import Window
w = Window.partitionBy('Snapshot Date', 'Checkin Date', 'DayDiff', 'Hotel Name', 'WeekDay')
hotelsBeforeClassifcation = hotels.withColumn('maxDis', f.max('DiscountPerc').over(w))\
    .where(f.col('DiscountPerc') == f.col('maxDis'))\
    .drop('maxDis')\
    .select('Snapshot Date', 'Checkin Date', 'DayDiff', 'Hotel Name', 'WeekDay', 'Discount Code')
hotelsBeforeClassifcation.show()

+-------------------+-------------------+--------------------+--------------------+---------+-------------+
|      Snapshot Date|       Checkin Date|             DayDiff|          Hotel Name|  WeekDay|Discount Code|
+-------------------+-------------------+--------------------+--------------------+---------+-------------+
|2015-07-17 00:00:00|2015-07-24 00:00:00|7 days 00:00:00.0...|DoubleTree by Hil...|   Friday|            3|
|2015-07-17 00:00:00|2015-08-09 00:00:00|23 days 00:00:00....|Hilton Garden Inn...|   Sunday|            1|
|2015-07-18 00:00:00|2015-08-06 00:00:00|19 days 00:00:00....|Eventi Hotel a Ki...| Thursday|            4|
|2015-07-18 00:00:00|2015-08-06 00:00:00|19 days 00:00:00....|Hilton Garden Inn...| Thursday|            3|
|2015-07-18 00:00:00|2015-08-06 00:00:00|19 days 00:00:00....|Hilton Garden Inn...| Thursday|            4|
|2015-07-18 00:00:00|2015-08-14 00:00:00|27 days 00:00:00....|Red Carpet Inn Ne...|   Friday|            1|
|2015-07-18 00:00:00|2015-08

In [106]:
hotelsBeforeClassifcation.coalesce(1).write.csv('Spark_Classification_Data', header=True)# Loading the data from csv

AnalysisException: 'path file:/C:/Git/FinalProjectDataScience/Spark_Classification_Data already exists.;'

In [125]:
hotelsForClassifcation = spark.read.csv('Spark_Classification_Data',inferSchema=True,header=True)

In [126]:
hotelsForClassifcation.show(5)

+-------------------+-------------------+-------+--------------------+--------+-------------+
|      Snapshot Date|       Checkin Date|DayDiff|          Hotel Name| WeekDay|Discount Code|
+-------------------+-------------------+-------+--------------------+--------+-------------+
|2015-07-17 00:00:00|2015-08-11 00:00:00|     25|     Conrad New York| Tuesday|            4|
|2015-07-18 00:00:00|2015-08-06 00:00:00|     19|Courtyard by Marr...|Thursday|            3|
|2015-07-18 00:00:00|2015-08-17 00:00:00|     30|The Peninsula New...|  Monday|            3|
|2015-07-20 00:00:00|2015-07-30 00:00:00|     10|W New York - Unio...|Thursday|            3|
|2015-07-20 00:00:00|2015-08-17 00:00:00|     28| Park Hyatt New York|  Monday|            2|
+-------------------+-------------------+-------+--------------------+--------+-------------+
only showing top 5 rows



In [127]:
temp = hotelsForClassifcation.limit(5)

In [128]:
temp.show(5)

+-------------------+-------------------+-------+--------------------+--------+-------------+
|      Snapshot Date|       Checkin Date|DayDiff|          Hotel Name| WeekDay|Discount Code|
+-------------------+-------------------+-------+--------------------+--------+-------------+
|2015-07-17 00:00:00|2015-08-11 00:00:00|     25|     Conrad New York| Tuesday|            4|
|2015-07-18 00:00:00|2015-08-06 00:00:00|     19|Courtyard by Marr...|Thursday|            3|
|2015-07-18 00:00:00|2015-08-17 00:00:00|     30|The Peninsula New...|  Monday|            3|
|2015-07-20 00:00:00|2015-07-30 00:00:00|     10|W New York - Unio...|Thursday|            3|
|2015-07-20 00:00:00|2015-08-17 00:00:00|     28| Park Hyatt New York|  Monday|            2|
+-------------------+-------------------+-------+--------------------+--------+-------------+



In [129]:
hotelsForClassifcation.show(1)
import pyspark.sql.functions as F
from pyspark.sql.functions import datediff
from pyspark.sql.functions import udf,unix_timestamp,col
from pyspark.sql.types import IntegerType,StringType

def weekToDay(x):
    try:
        return {
        'Sunday': 1,
        'Monday': 2,
        'Tuesday': 3,
        'Wednesday': 4,
        'Thursday': 5,
        'Friday': 6,
        'Saturday': 7
    }[x]
    except:
        return 8

#spark.udf.register("checkUdf",checkUdf)
#spark.udf.register("check",check)
#weekToDay_udf = udf(weekToDay, IntegerType())
hotelsForClassifcation.show(1)
spHotelsClassification = hotelsForClassifcation.withColumn('Snapshot Date', unix_timestamp(col('Snapshot Date'), "mm/dd/yyyy")). \
        withColumn('Checkin Date', unix_timestamp(hotelsForClassifcation['Checkin Date'], "mm/dd/yyyy"))
spHotelsClassification.show(5)
#newCol = checkUdf(spHotelsClassification["WeekDay"])
spHotelsClassification = spHotelsClassification.withColumn("WeekDay", F.ascii(col("WeekDay")))
spHotelsClassification.show(5)

from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Hotel Name", outputCol="Hotel Name_idx")
indexed = indexer.fit(spHotelsClassification).transform(spHotelsClassification)
final = indexed.drop("Hotel Name").withColumnRenamed("Hotel Name_idx", "Hotel Name")
final.show()

+-------------------+-------------------+-------+---------------+-------+-------------+
|      Snapshot Date|       Checkin Date|DayDiff|     Hotel Name|WeekDay|Discount Code|
+-------------------+-------------------+-------+---------------+-------+-------------+
|2015-07-17 00:00:00|2015-08-11 00:00:00|     25|Conrad New York|Tuesday|            4|
+-------------------+-------------------+-------+---------------+-------+-------------+
only showing top 1 row

+-------------------+-------------------+-------+---------------+-------+-------------+
|      Snapshot Date|       Checkin Date|DayDiff|     Hotel Name|WeekDay|Discount Code|
+-------------------+-------------------+-------+---------------+-------+-------------+
|2015-07-17 00:00:00|2015-08-11 00:00:00|     25|Conrad New York|Tuesday|            4|
+-------------------+-------------------+-------+---------------+-------+-------------+
only showing top 1 row

+-------------+------------+-------+--------------------+--------+------

In [130]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

vAssembler = VectorAssembler(inputCols= ['Snapshot Date','Checkin Date','Hotel Name','DayDiff','WeekDay'],outputCol="features")
vector = vAssembler.transform(final)
dataAndTarget = vector.select("features", "Discount Code")

train_data,test_data = dataAndTarget.randomSplit([0.7,0.3])
train_data.head()

Row(features=DenseVector([1437080400.0, 1437166800.0, 31.0, 1.0, 83.0]), Discount Code=2)

In [131]:
from pyspark.ml.classification import DecisionTreeClassifier

decisionTree = DecisionTreeClassifier(labelCol="Discount Code", featuresCol="features", maxBins=554, maxDepth=30)

dtmodel = decisionTree.fit(train_data)

dtpredictions = dtmodel.transform(test_data)
dtevaluator = MulticlassClassificationEvaluator(labelCol="Discount Code", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = dtevaluator.evaluate(dtpredictions)
print("Decision Tree test set accuracy = " + str(accuracy))

Decision Tree test set accuracy = 0.7664623368808996


In [114]:
from pyspark.ml.classification import NaiveBayes

navBay = NaiveBayes(modelType="multinomial", labelCol="Discount Code")

nbmodel = navBay.fit(train_data)

nbpredictions = nbmodel.transform(test_data)
nbevaluator = MulticlassClassificationEvaluator(labelCol="Discount Code", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = nbevaluator.evaluate(nbpredictions)
print("Naive bayes test set accuracy = " + str(accuracy))

Naive bayes test set accuracy = 0.19313656158112183


In [115]:
dtmodel.featureImportances

SparseVector(5, {0: 0.2852, 1: 0.2264, 2: 0.3639, 3: 0.1011, 4: 0.0234})

In [133]:
# Let's Calculate TP and FP
from pyspark.mllib.evaluation import MulticlassMetrics

decTree = dtpredictions.rdd.map(lambda x: (x["prediction"], float(x["Discount Code"])))
navBay = nbpredictions.rdd.map(lambda x: (x["prediction"], float(x["Discount Code"])))

metricsDT = MulticlassMetrics(decTree)
metricsNB = MulticlassMetrics(navBay)

print('True Positive - Decision Tree')
print(metricsDT.truePositiveRate(1.0),metricsDT.truePositiveRate(2.0),metricsDT.truePositiveRate(3.0),metricsDT.truePositiveRate(4.0))
print('True Positive - Naive Bayes')
print(metricsNB.truePositiveRate(1.0),metricsNB.truePositiveRate(2.0),metricsNB.truePositiveRate(3.0),metricsNB.truePositiveRate(4.0))
print('False Poisitive - Decision Tree')
print(metricsDT.falsePositiveRate(1.0),metricsDT.falsePositiveRate(2.0),metricsDT.falsePositiveRate(3.0),metricsDT.falsePositiveRate(4.0))
print('False Poisitive - Naive Bayes')
print(metricsNB.falsePositiveRate(1.0),metricsNB.falsePositiveRate(2.0),metricsNB.falsePositiveRate(3.0),metricsNB.falsePositiveRate(4.0))

True Positive - Decision Tree


Py4JJavaError: An error occurred while calling o4681.truePositiveRate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1038.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1038.0 (TID 2665, localhost, executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:170)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:97)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:108)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:324)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:288)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrRea

In [ ]:
print('True Positive - Naive Bayes')
print(metricsNB.truePositiveRate(1.0),metricsNB.truePositiveRate(2.0),metricsNB.truePositiveRate(3.0),metricsNB.truePositiveRate(4.0))
print('False Poisitive - Decision Tree')
print(metricsDT.falsePositiveRate(1.0),metricsDT.falsePositiveRate(2.0),metricsDT.falsePositiveRate(3.0),metricsDT.falsePositiveRate(4.0))
print('False Poisitive - Naive Bayes')
print(metricsNB.falsePositiveRate(1.0),metricsNB.falsePositiveRate(2.0),metricsNB.falsePositiveRate(3.0),metricsNB.falsePositiveRate(4.0))